# import libraries

In [13]:
import pandas as pd

# ignore warning 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/tmp/ipykernel_13634/2921450178.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# upload data

In [2]:
df1 = pd.read_json('/fastcampus-data/select_column_version_3.json')
df = df1.copy()

In [3]:
# log data의 기간(4개월)과 동일한 기간의 데이터 사용하기 위해 start date 지정 
start_date = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]

# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > start_date]

# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
complete_df = complete_df.reset_index(drop=True)

# 도서 카테고리만 가져오기
df_book = complete_df[complete_df['name'] == '도서']
df_book = df_book.sort_values(by='date_paid')


# train test split
최신성을 반영하기 위해 전체 데이터의 기간(4개월) 중 마지막 3주의 데이터를 test로 사용했다.


4개월의 데이터를 활용하여 3주 이전의 데이터로 학습하고 마지막 3주를 예측한다. 

In [4]:
from datetime import datetime, timedelta

# 마지막 3주를 test data로 사용
date = df_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

# train 1개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=1].index
train = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]

In [5]:
# 전처리할 test index 확인: 2번 이상 구매한 고객만 추출
drop_row_index = test_before_preprocess['customer_id'].value_counts()[test_before_preprocess['customer_id'].value_counts()>2].index

# 최근 구매 3개 기록을 test 사용
test_list = []
for idx in drop_row_index:
    test_list.append(test_before_preprocess[test_before_preprocess['customer_id']==idx].iloc[:])
test = pd.concat(test_list)

# train 있는 유저에 대해서만 진행
test = test[test['customer_id'].isin(train['customer_id'])]

In [6]:
train.date_paid = pd.to_datetime(train.date_paid)

In [7]:
# date_paid에 날짜만 남기기
train.date_paid = train.date_paid.apply(lambda x: x.date)

# Load Transactions

In [8]:
train = train[['date_paid','customer_id','name_x','product_ids']]
print('Train shape:',train.shape)

Train shape: (5042, 4)


In [9]:
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
train_apri[:30]

,customer_id,name_x
0,5d60cab24e77525ec5ca13d5,"[흔히보는 정형외과 외래진료 가이드북, 숲을 보는 요통치료]"
1,5d60cb754e77525ec5ca13d9,[흔히보는 정형외과 외래진료 가이드북]
2,5d60cc294e77525ec5ca13e0,[약침의 정석 –통증편]
3,5d60ccd84e77525ec5ca13f2,[증보운곡본초학]
4,5d60cce84e77525ec5ca13f3,[보험한약 브런치 the # 2판 개정판]
5,5d60cd1b4e77525ec5ca13fd,"[윤상훈·권병조의 알짜 근육학, 동씨침법입문 2판]"
6,5d60cd414e77525ec5ca1401,"[흔히보는 정형외과 외래진료 가이드북, 약침의 정석 –통증편, 초음파 유도하 침 시..."
7,5d60cd634e77525ec5ca1402,"[트리거포인트 침치료, 흔히보는 정형외과 외래진료 가이드북, 뇌의학의 첫걸음]"
8,5d60ce694e77525ec5ca140e,[趙紹琴(조소금) 내과학]
9,5d60cea34e77525ec5ca1413,"[흔히보는 정형외과 외래진료 가이드북, 비만문답]"


In [10]:
from mlxtend.preprocessing import TransactionEncoder

In [14]:
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id','date_paid']))]

In [15]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Convert product list into one-hot

In [16]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

describe를 이용하여 most popular item 확인

In [17]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent : f가 작을 수록 판매된 횟수가 많은 것
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

MOST POPULAR


,Count,Item
84,575,비만문답
211,448,플로차트 정형외과 진단
242,315,흔히보는 정형외과 외래진료 가이드북
144,223,윤상훈·권병조의 알짜 근육학
241,213,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼
240,197,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
131,143,외래에서 꼭 알아야 할 통증증후군 137가지
196,134,카이로프랙틱 기본테크닉론
90,125,사암침의 해석과 임상
126,101,약침의 정석 –통증편


# Apriori

In [18]:
from mlxtend.frequent_patterns import apriori

In [21]:
is_ap = apriori(orders_1hot, min_support=0.01, max_len=3, use_colnames=True)
is_ap.sort_values(by = ['support'], ascending = False)

,support,itemsets
4,0.156804,(비만문답)
21,0.122171,(플로차트 정형외과 진단)
25,0.085901,(흔히보는 정형외과 외래진료 가이드북)
12,0.060813,(윤상훈·권병조의 알짜 근육학)
24,0.058086,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼)
23,0.053722,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록)
27,0.048268,"(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록, 황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼)"
11,0.038996,(외래에서 꼭 알아야 할 통증증후군 137가지)
19,0.036542,(카이로프랙틱 기본테크닉론)
6,0.034088,(사암침의 해석과 임상)


# Association Rules
confidence로 threshold를 설정하여 filtering한 다음 lift값을 기준으로 sorting한다.

In [22]:
from mlxtend.frequent_patterns import association_rules

# matrix의 default가 confidence로 되어있어 따로 지정해주지 않아도 됨.
rules_confidence_item = association_rules(is_ap, min_threshold=0.04)

#lift 기준으로 sort
rules_confidence_item = rules_confidence_item.sort_values(['lift'], ascending = False)
display(rules_confidence_item)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.053722,0.058086,0.048268,0.898477,15.468149,0.045148,9.277857
3,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.058086,0.053722,0.048268,0.830986,15.468149,0.045148,5.598809
0,(사암침의 해석과 임상),(비만문답),0.034088,0.156804,0.026179,0.768000,4.897837,0.020834,3.634466
1,(비만문답),(사암침의 해석과 임상),0.156804,0.034088,0.026179,0.166957,4.897837,0.020834,1.159498


### 결론: confidence가 높은 association rule의 개수가 적다.
-> 연관성 추천이 불가능하다.

# pred와 test의 real order list 비교
* prediction: association rule의 antecedents item을 구매한 유저가 consequents item을 구매할 것
* consequents item을 예측값으로 하고 test의 실데 유저가 구매한 도서가 예측한 도서와 일치하는지 비교

In [23]:
apriori_items = rules_confidence_item[['antecedents','consequents']].iloc[:15]

apriori_items['antecedents'] = apriori_items['antecedents'].apply(lambda x: list(map(str,x))[0])
apriori_items['consequents'] = apriori_items['consequents'].apply(lambda x: list(map(str,x))[0])

apriori_items.rename(columns={"antecedents": "A", "consequents": "A의 연관 아이템 (B)"})

,A,A의 연관 아이템 (B)
2,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼
3,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
0,사암침의 해석과 임상,비만문답
1,비만문답,사암침의 해석과 임상


In [24]:
train_items = train[['date_paid','customer_id','name_x','product_ids']]

# 예측값을 담을 column
train_items['apriori_pred'] = train_items['name_x'].apply(lambda x: apriori_items[apriori_items['antecedents']==x]['consequents'].values[0] if x in list(apriori_items['antecedents']) \
                            else None)

In [25]:
name_x_df = train_items.groupby('customer_id')['name_x'].unique().reset_index()
apriori_pred_df  = train_items.groupby('customer_id')['apriori_pred'].unique().reset_index()
real_pred = test.groupby('customer_id')['name_x'].unique().reset_index().rename(columns={'name_x':'real_product'})

train_pred = pd.merge(name_x_df,apriori_pred_df, on='customer_id', how='left')
train_real_pred = pd.merge(train_pred,real_pred, on='customer_id', how='right')

train_real_pred[['name_x','apriori_pred','real_product']]

,name_x,apriori_pred,real_product
0,[플로차트 정형외과 진단],[None],"[미안침, 파킨슨병 한의진료, 필라테스 해부학(PILATES ANATOMY)]"
1,"[기본통증진료학, 최수용의 기능의학 해설, 약침의 정석 –통증편]",[None],"[뇌의학의 첫걸음, 한의학 상담, 수면장애의 한방치료]"
2,"[플로차트 정형외과 진단, 외래에서 꼭 알아야 할 통증증후군 137가지]",[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 비만문답]"
3,"[윤상훈·권병조의 알짜 근육학, 그린만의 수기의학 원리]",[None],"[임상 한의사를 위한 기본 한약처방 강의 2판, 플로차트 정형외과 진단, 장골의 PI 변위는 없다]"
4,"[황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼, 황황교수의 개원 한의사를 위한 상한금궤 처방 강의록, 플로차트 정형외과 진단]","[황황교수의 개원 한의사를 위한 상한금궤 처방 강의록, 황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼, None]","[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 카이로프랙틱 기본테크닉론]"
5,"[파킨슨병 한의진료, 우리 아이 감기, 이해하면 보이는 어깨치료 ABC, 비수술정형외과, 무릎통증(아프니까 무릎이다)]",[None],"[감별진단의 정석, 신경학 증상의 감별법, The 정형내과(The Orthopaedic Medicine), 초음파 유도하 침 시술 가이드북, 장골의 PI 변위는 없다]"
6,"[카이로프랙틱 기본테크닉론, 흔히보는 정형외과 외래진료 가이드북, 趙紹琴(조소금) 내과학, 안면마비 침구치료, 한의학 상담, 중경서 독법 강해(상,하) /개정판, 의학심오(醫學心悟), 숨찬 세상, 호흡기를 편하게, 황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼, 황황교수의 개원 한의사를 위한 상한금궤 처방 강의록, 갑상선 진료 완전정복, HAPPY 소아청소년 진료, 신경학 증상의 감별법, 이것이 알고싶다! 당뇨병진료, 어지럼질환의 진단과 치료, 증례와 함께 하는 한약처방, 약침의 정석 –통증편, 실전, 임상한의학 내과질환을 중심으로, 평주온열경위, 실전, 임상한의학 알레르기질환, 침구대성, 내과학 5권세트, 한방순환 신경내과학, 사암침의 해석과 임상, 플로차트 정형외과 진단, 외래에서 꼭 알아야 할 통증증후군 137가지, SMART 기본 일차진료매뉴얼 3판(세트), SMART 응급진료매뉴얼(세트), SMART 소아진료매뉴얼 3판]","[None, 황황교수의 개원 한의사를 위한 상한금궤 처방 강의록, 황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼, 비만문답]","[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 소아피부질환해설]"
7,"[이것이 알고싶다! 당뇨병진료, 그림으로 이해하는 수분과 전해질 및 산-염기 대사, 갑상선 진료 완전정복]",[None],"[초음파 유도하 침 시술 가이드북, SMART 응급진료매뉴얼(세트), 영어 진료 가이드북 ]"
8,"[비만문답, 플로차트 정형외과 진단, 침의 과학적 접근의 이해, 근골격계 약침의학, 말초신경 약침의학, 한의 피부진료 첫 걸음]","[사암침의 해석과 임상, None]","[임상 한의사를 위한 기본 한약처방 강의 2판, 일차진료 한의사를 위한 보험한약입문 - 둘째 판, 숲을 보는 요통치료, NEO 인턴 핸드북]"
9,"[플로차트 정형외과 진단, 초음파 유도하 침 시술 가이드북]",[None],"[비만문답, 바른 자세를 위한 교정운동, 약침의 정석 –통증편]"


In [26]:
# 아이템 맞춘 개수
cnt = 0
for preds, reals in zip(train_real_pred['apriori_pred'], train_real_pred['real_product']):
    for pred in preds:
        if pred in reals:
            cnt += 1
print('예측 성공한 아이템 개수: ',cnt)

예측 성공한 아이템 개수:  0


### 결론: 
* apriori분석은 전체 구매에서 해당 아이템의 출현 빈도를 기반으로 연관분석을 하는 것이다. 도서 데이터로 apriori분석을 한 결과, 각 도서의 support가 너무 낮다. 즉, 도서를 구매한 빈도가 낮아 연관분석을 진행하기 어렵다.
* consequents item을 예측값으로 하고 test의 실제 구매한 item과 비교했을 때 일치하는 것이 없었다. antecedents item과 consequents item이 함께 출현한 정도를 나타내는 지표인 confidence의 min_treshold의 값을 더 낮추어 association rule의 개수를 늘릴 수도 있지만, confidence가 낮다면 해당 item들이 함께 구매되지 않는 것이기 때문에 연관성이 낮은 rule들이 늘어난다.
* 해당 데이터셋으로는 도서 구매 빈도가 낮아 데이터 부족하기 때문에 연관성 분석 및 추천이 불가능하다.